In [ ]:
!pip install tenacity

In [ ]:
import os
import glob
import json
import sys
from datetime import datetime
from tenacity import retry, stop_after_attempt, wait_fixed, retry_if_exception_type

# ==========================================
# [필수] Colab 드라이브 마운트 및 설정
# ==========================================

# 1. Colab 환경 감지 및 드라이브 마운트
try:
    from google.colab import drive
    print("[시스템] Colab 환경 감지됨. 드라이브를 마운트합니다...")

    # 이미 마운트 되어있지 않다면 마운트 수행
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    IS_COLAB = True

except ImportError:
    IS_COLAB = False
    print("[시스템] 로컬 환경입니다. (구글 드라이브 파일 스트림 등이 설치되어 있어야 작동함)")

# ==========================================
# 사용자 설정 영역 (이 부분을 꼭 확인하세요!)
# ==========================================

# [중요] 1. 구글 드라이브 웹에서 공유 폴더를 '내 드라이브에 바로가기 추가' 하세요.
# [중요] 2. 추가한 바로가기 폴더의 정확한 이름을 아래 변수에 입력하세요.
# 예: 내 드라이브에 'LOL_DATA'라는 이름으로 바로가기를 만들었다면 'LOL_DATA' 입력
TARGET_SHORTCUT_NAME = 'telemetry'

# 경로 설정
if IS_COLAB:
    # 바로가기를 통해 접근할 원본 데이터 경로 (읽기 전용으로 사용)
    # 147GB 데이터를 다운로드하지 않고, 여기서 직접 하나씩 엽니다.
    SOURCE_DATA_DIR = f'/content/drive/MyDrive/{TARGET_SHORTCUT_NAME}'

    # 결과를 저장할 경로 (내 드라이브)
    BASE_RESULT_DIR = '/content/drive/MyDrive/pubg_telemetry_summary'
else:
    # 로컬 테스트용 경로
    SOURCE_DATA_DIR = f'./{TARGET_SHORTCUT_NAME}'
    BASE_RESULT_DIR = './extracted_results'

# 결과 파일 저장 폴더가 없으면 생성
if not os.path.exists(BASE_RESULT_DIR):
    os.makedirs(BASE_RESULT_DIR)

# [필터 설정]
# 특정 이벤트만 뽑고 싶다면 여기에 추가 (예: ["LogPlayerKill"])
TARGET_EVENTS = ["LogGameStatePeriodic",'LogMatchStart','LogParachuteLanding','LogPhaseChange','LogPlayerKillV2',
                 'LogPlayerPosition','LogItemPickup', 'LogItemEquip', 'LogItemAttach','LogItemPickupFromCarepackage','LogItemPickupFromLootbox'
                 'LogVehicleRide','logvehicleleave','LogMatchEnd']

@retry(stop=stop_after_attempt(5), wait=wait_fixed(2), retry=retry_if_exception_type(OSError))
def process_json_file(file_path):
    """
    JSON 파일을 열어서 필요한 정보를 추출합니다.
    다운로드 없이 드라이브에서 직접 스트리밍으로 읽으므로 용량을 차지하지 않습니다.
    또한, LogMatchStart 이벤트에서 맵 이름과 매치 날짜/시간을 추출합니다.
    """
    map_name = None
    match_datetime_str = None
    team_size_str = 'unknown_team' # 기본값 설정

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not isinstance(data, list):
            return [], [], map_name, match_datetime_str, team_size_str

        # 이벤트 타입 수집 (디버깅용)
        unique_event_types = list(set(item.get('_T', 'Unknown') for item in data if isinstance(item, dict)))

        # LogMatchStart 이벤트에서 맵 이름, 시간, 팀 크기 추출
        for item in data:
            if item.get('_T') == 'LogMatchStart':
                map_name = item.get('mapName')
                _D_raw = item.get('_D')
                if _D_raw:
                    try:
                        # '2024-05-15T12:34:56.789Z' 형식 처리
                        dt_obj = datetime.strptime(_D_raw.split('.')[0], '%Y-%m-%dT%H:%M:%S')
                        match_datetime_str = dt_obj.strftime('%Y%m%d_%H%M%S')
                    except ValueError:
                        pass # 날짜 파싱 실패 시 None 유지

                team_size = item.get('teamSize')
                if team_size == 1:
                    team_size_str = 'solo'
                elif team_size == 2:
                    team_size_str = 'duo'
                elif team_size == 4:
                    team_size_str = 'squad'
                else:
                    team_size_str = f'team_{team_size}' if team_size is not None else 'unknown_team'

                break # 첫 번째 LogMatchStart만 사용

        # 필터링 로직
        if TARGET_EVENTS:
            filtered_logs = [item for item in data if item.get('_T') in TARGET_EVENTS]
        else:
            filtered_logs = data

        return filtered_logs, unique_event_types, map_name, match_datetime_str, team_size_str

    except json.JSONDecodeError:
        print(f"[오류] JSON 파싱 불가: {os.path.basename(file_path)}")
        return [], [], map_name, match_datetime_str, team_size_str
    except OSError as e:
        print(f"[오류] 파일 읽기 재시도: {os.path.basename(file_path)} - {e}")
        raise # Re-raise to trigger tenacity retry
    except Exception as e:
        print(f"[오류] 파일 읽기 실패: {e}")
        return [], [], map_name, match_datetime_str, team_size_str

@retry(stop=stop_after_attempt(5), wait=wait_fixed(2), retry=retry_if_exception_type(OSError))
def save_match_logs(output_path, logs):
    """
    주어진 로그 리스트를 지정된 경로에 JSONL 형식으로 저장합니다.
    OSError 발생 시 재시도합니다.
    """
    with open(output_path, 'w', encoding='utf-8') as output_handle:
        for log in logs:
            output_handle.write(json.dumps(log, ensure_ascii=False) + '\n')

def main():
    print("=== 구글 드라이브 대용량 데이터 직접 분석기 ===")
    print(f"대상 폴더 경로: {SOURCE_DATA_DIR}")

    # 폴더 존재 여부 확인 (바로가기 이름이 틀렸을 경우 대비)
    if not os.path.exists(SOURCE_DATA_DIR):
        print(f"\n[오류] '{SOURCE_DATA_DIR}' 경로를 찾을 수 없습니다.")
        print("1. 구글 드라이브에서 공유 폴더를 '내 드라이브에 바로가기 추가' 했는지 확인하세요.")
        print("2. 코드 상단의 'TARGET_SHORTCUT_NAME' 변수가 바로가기 이름과 정확히 일치하는지 확인하세요.")
        return

    print("\n[탐색 시작] 드라이브 내의 JSON 파일을 찾습니다...")
    # recursive=True로 하위 폴더까지 모두 탐색
    json_files = glob.glob(os.path.join(SOURCE_DATA_DIR, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"폴더 내에 JSON 파일이 없습니다. 경로를 다시 확인해주세요: {SOURCE_DATA_DIR}")
        return

    print(f"총 {len(json_files)}개의 파일을 발견했습니다.")
    print("다운로드 없이 직접 분석을 시작합니다. (속도는 인터넷 연결 상태에 따라 다름)")

    # 상태 변수
    processed_count = 0
    total_logs_saved = 0
    first_file_events = None

    try:
        for file_path in json_files:
            # 파일 이름에서 매치 ID 추출 (확장자 제거), 기본값으로 사용
            original_match_id = os.path.splitext(os.path.basename(file_path))[0]

            try:
                logs, event_types, map_name, match_datetime_str, team_size_str = process_json_file(file_path)
            except Exception as e:
                print(f"[치명적 오류] 파일 처리 실패 후 재시도 종료: {os.path.basename(file_path)} - {e}")
                continue # 재시도 실패 시 다음 파일로 건너뛰기

            # 매치 ID 생성 (맵 이름과 시간 정보 우선 사용)
            parts = []
            if match_datetime_str:
                parts.append(match_datetime_str)
            if team_size_str != 'unknown_team': # 팀 크기가 유효하면 추가
                parts.append(team_size_str)
            if map_name: # 맵 이름도 유효하면 추가
                sanitized_map_name = map_name.replace(' ', '_').replace('/', '_').replace('\\', '_')
                parts.append(sanitized_map_name)

            if parts: # 조합된 정보가 있으면 사용
                match_id = '_'.join(parts)
            else: # 조합된 정보가 없으면 원본 파일 이름 사용
                match_id = original_match_id

            # 매치별 결과 파일 경로 생성
            match_output_path = os.path.join(BASE_RESULT_DIR, f"{match_id}.jsonl")

            # 첫 번째 파일 분석 결과 출력 (가이드)
            if not first_file_events and event_types:
                first_file_events = event_types
                print(f"\n[정보] 발견된 이벤트 태그 예시:")
                print(f"{first_file_events[:15]} ... (총 {len(first_file_events)}종)")
                if not TARGET_EVENTS:
                    print(">> 현재 필터 없이 [모든 로그]를 저장 중입니다.")
                print("-" * 50)

            if logs:
                try:
                    save_match_logs(match_output_path, logs)
                    total_logs_saved += len(logs) # 성공적으로 저장된 로그 수 업데이트
                except Exception as e:
                    print(f"[치명적 오류] 로그 저장 실패 후 재시도 종료: {os.path.basename(file_path)} -> {os.path.basename(match_output_path)} - {e}")
                    # (선택 사항) 실패한 파일 경로를 기록하여 나중에 재처리할 수 있습니다.
                    continue
            else:
                # 필터링된 로그가 없는 경우 (선택 사항: 경고 메시지 등)
                pass

            processed_count += 1
            if processed_count % 10 == 0:
                print(f"진행률: {processed_count}/{len(json_files)} 파일 완료 | 누적 {total_logs_saved}건 | 현재 파일: {os.path.basename(file_path)}", end='\r')

    finally:
        # 이제 전역 output_handle이 없으므로 닫을 필요 없음
        pass

    print(f"\n\n[작업 완료] 총 {processed_count}개 파일 처리됨.")
    print(f"결과는 '{BASE_RESULT_DIR}' 폴더에 개별 매치 파일로 저장되었습니다.")

if __name__ == '__main__':
    main()

[시스템] Colab 환경 감지됨. 드라이브를 마운트합니다...
=== 구글 드라이브 대용량 데이터 직접 분석기 ===
대상 폴더 경로: /content/drive/MyDrive/telemetry

[탐색 시작] 드라이브 내의 JSON 파일을 찾습니다...
총 6757개의 파일을 발견했습니다.
다운로드 없이 직접 분석을 시작합니다. (속도는 인터넷 연결 상태에 따라 다름)

[정보] 발견된 이벤트 태그 예시:
['LogMatchStart', 'LogVehicleRide', 'LogMatchDefinition', 'LogCarePackageSpawn', 'LogMatchEnd', 'LogParachuteLanding', 'LogCarePackageLand', 'LogPlayerDestroyProp', 'LogItemEquip', 'LogItemPickup', 'LogPlayerKillV2', 'LogPhaseChange', 'LogVehicleDestroy', 'LogPlayerLogin', 'LogGameStatePeriodic'] ... (총 45종)
--------------------------------------------------


[작업 완료] 총 6757개 파일 처리됨.
결과는 '/content/drive/MyDrive/pubg_telemetry_summary' 폴더에 개별 매치 파일로 저장되었습니다.
